# Módulo Deep Learning
## Actividad 2: Reinforcement Learning: **Frozen lake problem**

-Simeon Milenov Mitev

-Daniel Gonzalez Valera

-Cristopher Pazmiño


# Actividad Reinforcemente Learning

Resolver el problema del Frozen lake de OpenAI Gym. Documentación: https://www.gymlibrary.dev/environments/toy_text/frozen_lake/

## Objetivos
- Conseguir movermos aleatoriamente hasta cumplir el objetivo
- Conseguir que el agente aprenda con Q-learning
- (Opcional) Probar con otros hiperparámetros
- (Opcional) Modificar la recompensa

## Consideraciones
- No hay penalizaciones
- Si el agente cae en un "hole", entonces done = True y se queda atascado sin poder salir (al igual que ocurre cuando llega al "goal")

## Normas a seguir

- Se debe entregar un **ÚNICO GOOGLE COLAB notebook** (archivo .ipynb) que incluya las instrucciones presentes y su **EJECUCIÓN!!!**.
- Poner el nombre del grupo en el nombre del archivo y el nombre de todos los integrantes del grupo al inicio del notebook.

## Criterio de evaluación

- Seguimiento de las normas establecidas en la actividad.
- Corrección en el uso de algoritmos, modelos y formas idiomáticas en Python.
- El código debe poder ejecutarse sin modificación alguna en Google Colaboratory.

## **Instalamos librerías**

In [1]:
!pip install gym==0.17.3
!pip install numpy==1.23.5

# Definimos las Librerías

In [19]:
import gym
import numpy as np
from time import sleep
from IPython.display import clear_output
import random as rd

##**Definición del entorno**

In [20]:
# Definimos el entorno
env = gym.make('FrozenLake-v0', desc=None, map_name="4x4", is_slippery=False)

In [21]:
# Fijamos una semilla
seed_value = 42
env.seed(seed_value)
np.random.seed(seed_value)

In [22]:
env.reset() # En este caso, empieza desde la misma posición inicial
print(env.render())


SFFF
FHFH
FFFH
HFFG
None


In [23]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(4)
State Space Discrete(16)


Acciones posibles:
* 0: izquierda
* 1: abajo
* 2: derecha
* 3: arriba

In [24]:
# Identificador de estado
state = env.s
print("State:", state)

State: 0


## **¡Nos movemos aleatoriamente!**

In [25]:
steps = 0
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [26]:
# Acciones: 0=izquierda, 1=abajo, 2=derecha, 3=arriba
action = 1
state, reward, done, info = env.step(action)

print("State:", state)
print(state, reward, done, info)

env.s = state
env.render()

steps += 1

print(f"Step: {steps}")

State: 4
4 0.0 False {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG
Step: 1


# Movimientos aleatorios hasta la recompensa

In [27]:
class bcolors:
    RED = '\u001b[31m'
    GREEN = '\u001b[32m'  # Para visualizar mejor los resultados como recompensas negativas o positivas.
    RESET = '\u001b[0m'

In [28]:
def run_frozen_lake_random_move_simulation():

    timestep, penalties, reward = 0, 0, 0
    done = False

    while reward == 0:
        if done is True:  # Definimos esta condición necesaria que se nos pide en el ejercicio.
            env.reset()

        action = env.action_space.sample() # Se elige una acción aleatoria.
        state, reward, done, info = env.step(action)  # Para ejecutar esa acción.

        timestep += 1

        # Limpia la salida y muestra el entorno después de cada acción.
        clear_output(wait=True)
        print("Estado del juego:")
        env.render(mode='human')
        print("")

        # Para imprimir los resultados.
        if reward < 0:
            print(f"Recompensa actual: {bcolors.RED}{reward}{bcolors.RESET}")
        else:
            print(f"Recompensa actual: {bcolors.GREEN}{reward}{bcolors.RESET}")
        print("")
        print('Estado actual', state)
        sleep(0.05)  # Tiempo de espera entre cada intento.

    # Ver los resultados.
    print("Pasos totales tomados: {}".format(timestep))
    print("Penalizaciones incurridas: {}".format(penalties)) # Para confirmar de que no hay penalizaciones.

In [30]:
run_frozen_lake_random_move_simulation()

Estado del juego:
  (Right)
SFFF
FHFH
FFFH
HFFG

Recompensa actual: 1.0

Estado actual 15
Pasos totales tomados: 237
Penalizaciones incurridas: 0


# Q-learning

In [31]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [32]:
q_table[14]

array([0., 0., 0., 0.])

In [33]:
def greedy(epsilon,q_table,state,env):
    if rd.random() < epsilon:
        action=env.action_space.sample()
    else:
        action=np.argmax(q_table[state])
    return action

In [34]:
# Hyperparameters
alpha = 0.1 # tasa de aprendizaje
gamma = 0.6 # tasa de descuento
epsilon = 0.1 # greedy policy

# For plotting metrics
all_timestep = []
all_penalties = []

episodes = 10001

for i in range(episodes):
    state = env.reset()

    timestep, penalties, reward = 0, 0, 0
    done = False

    while reward == 0:

        if done is True:
            env.reset()

        action = greedy(epsilon,q_table,state,env)

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action] # En la Q-table, tomamos el valor Q de la acción elegida para el estado actual.
        next_max = np.max(q_table[next_state]) # En la Q-table, tomamos el máximo entre los valores Q para el nuevo estado.

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max) # Actualizamos el valor Q.
        q_table[state, action] = new_value

        state = next_state
        timestep += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 10000
Training finished.



In [36]:
q_table[14]

array([0.36, 0.6 , 1.  , 0.36])

In [37]:
class bcolors:
    RED= '\u001b[31m'
    GREEN= '\u001b[32m'
    RESET= '\u001b[0m'

env.s = 0
done = False

timestep, penalties, reward = 0, 0, 0
total_reward = 0

while reward == 0:

    if done is True:
            env.reset()

    action = np.argmax(q_table[state]) # Se aplica lo aprendido.
    state, reward, done, info = env.step(action)

    timestep += 1
    total_reward += reward

  # Para imprimir cada paso
    clear_output(wait=True)
    env.render()
    print("")
    if reward < 0:
        print(f"Recompensa actual: {bcolors.RED}{reward}{bcolors.RESET}")
    else:
        print(f"Recompensa actual: {bcolors.GREEN}{reward}{bcolors.RESET}")
    print("")
    print('Estado actual', state)
    sleep(0.05)


print("Timesteps taken: {}".format(timestep))
print("Penalties incurred: {}".format(penalties))

  (Right)
SFFF
FHFH
FFFH
HFFG

Recompensa actual: 1.0

Estado actual 15
Timesteps taken: 8
Penalties incurred: 0
